In [ ]:
# group by node id (since rows are currently node events) .mean of is_crash to have dummy danger metric
# associate the node ids with user traslation table to have node ID with tract ID then associate with demographic data
# linear regression: ex: percentage black with crash ratio

In [2]:
import pandas as pd

In [3]:
# load node events
node_events_df = pd.read_csv("../../data/unified_dataset.csv")

In [5]:
# check shape
node_events_df.shape

(22594153, 42)

In [4]:
# examine head
node_events_df.head()

,Unnamed: 0,NODE_ID,NODE_LATITUDE,NODE_LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,...,WT09,WT10,WT11,WT13,WT14,WT15,WT16,WT18,WT19,WT22
0,0,42430108,40.724267,-74.004739,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,42424864,40.724413,-74.005930,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,42423456,40.723791,-74.006113,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4602414021,40.723761,-74.006117,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,42452015,40.723132,-74.006215,0.0,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# check columns
node_events_df.columns

Index(['Unnamed: 0', 'NODE_ID', 'NODE_LATITUDE', 'NODE_LONGITUDE',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'EVENT_DIST_FROM_NODE', 'EVENT_DATE', 'EVENT_TIME', 'IS_CRASH', 'DATE',
       'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'WT01',
       'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT10', 'WT11',
       'WT13', 'WT14', 'WT15', 'WT16', 'WT18', 'WT19', 'WT22'],
      dtype='object')

In [20]:
# create df with node ID and IS_CRASH
node_ids_crash_avg = node_events_df[['NODE_ID','IS_CRASH']]
node_ids_crash_avg.head()

,NODE_ID,IS_CRASH
0,42430108,False
1,42424864,False
2,42423456,False
3,4602414021,False
4,42452015,False


In [21]:
# group by node ID and take average of IS_CRASH to have toy danger metric
node_id_crash_df = node_ids_crash_avg.groupby(by='NODE_ID').agg('mean')

In [45]:
# examine head
node_id_crash_df.head()

,IS_CRASH
NODE_ID,
42421828,0.000000
42421837,0.000000
42421877,0.000272
42421889,0.000032
42421927,0.000000


In [44]:
# check unique NOIDE_ID in df matches number of rows in node_id_crash_df
assert len(set(node_events_df['NODE_ID'])) == node_id_crash_df.shape[0]

# check how many unique NODE_ID in original df
len(set(node_events_df['NODE_ID']))

2101

In [35]:
# load node id to census tract key table
node_id_census_tract = pd.read_csv("../../data/node_id_census_tract_key.csv")

# check shape
node_id_census_tract.shape

2587

In [46]:
# examine head
node_id_census_tract.head()

,osmid,ct2010
0,42421828,2800
1,42421837,2800
2,42430924,2800
3,42430938,2800
4,42436957,2800


In [56]:
# look at why there are more rows in census 
# ASSUMING THESE ARE NODES FOR WHICH WE HAVE NO NODE EVENTS (neither crashes nor citiBike)
print("number of node ids in census tract df not in crash df: ", len(set(node_id_census_tract['osmid'])) - len(set(node_events_df['NODE_ID'])))

node_id_crash_df.columns

number of node ids in census tract df not in crash df:  486


Index(['IS_CRASH'], dtype='object')

In [80]:
# merge on node ids: census tracts and avg crash
node_ids__crash_avg_census_tracts = node_id_crash_df.merge(node_id_census_tract, how = 'left', left_index=True, right_on='osmid')

# rename columns to be clearer
col_names = {'osmid':'NODE_ID','IS_CRASH':'CRASH_AVG','ct2010':'CENSUS_TRACT_ID'}
node_ids__crash_avg_census_tracts.rename(columns=col_names, inplace=True)

# check shape
node_ids__crash_avg_census_tracts.shape

(2101, 3)

In [81]:
# examine head
node_ids__crash_avg_census_tracts.head()

,CRASH_AVG,NODE_ID,CENSUS_TRACT_ID
0,0.000000,42421828,2800
1,0.000000,42421837,2800
27,0.000272,42421877,6300
52,0.000032,42421889,7100
72,0.000000,42421927,5502


In [82]:
node_ids__crash_avg_census_tracts.sort_values('CRASH_AVG', ascending=False)

,CRASH_AVG,NODE_ID,CENSUS_TRACT_ID
171,1.0,42431902,900
123,1.0,42429507,700
2578,1.0,370893397,2000
970,1.0,4500012797,5400
557,1.0,7137281959,6200
...,...,...,...
2404,0.0,42456568,7200
1260,0.0,42456303,5900
1259,0.0,42456211,5900
1258,0.0,42456206,5900


In [104]:
print("Nodes with no Crashes: number of nodes (# CRASH / # EVENTS) == 0 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']==0))
# there are Crash Averages ==1 so looking into this:
## do we want to remove Crash Averages that are above a certain fraction?
## for example, greater than .001 meaning that there was one crash in 1000 events for the node

print("number of nodes (# CRASH / # EVENTS) == 1 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']==1))
print("number of nodes (# CRASH / # EVENTS) > 1 / 10 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.1))
print("number of nodes (# CRASH / # EVENTS) > 1 / 100 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.01))
print("number of nodes (# CRASH / # EVENTS) > 1 / 1000 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.001))

Nodes with no Crashes: number of nodes (# CRASH / # EVENTS) == 0 :  1005
number of nodes (# CRASH / # EVENTS) == 1 :  29
number of nodes (# CRASH / # EVENTS) > 1 / 10 :  29
number of nodes (# CRASH / # EVENTS) > 1 / 100 :  36
number of nodes (# CRASH / # EVENTS) > 1 / 1000 :  125


In [120]:
# look at sample CRASH EVENT == 1 node from original df
node_events_df.loc[node_events_df['NODE_ID']==1728266963,:]

,Unnamed: 0,NODE_ID,NODE_LATITUDE,NODE_LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,...,WT09,WT10,WT11,WT13,WT14,WT15,WT16,WT18,WT19,WT22
22591956,22591956,1728266963,40.736698,-73.974499,1.0,0.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22592078,22592078,1728266963,40.736698,-73.974499,1.0,0.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
# look at rows with Crash Avg == 1 to ignore in correlation
crash_avg_equals_one = node_ids__crash_avg_census_tracts.loc[node_ids__crash_avg_census_tracts['CRASH_AVG']==1,:]

,CRASH_AVG,NODE_ID,CENSUS_TRACT_ID
657,1.0,42424382,3300
1474,1.0,2496650741,800
1771,1.0,7631032411,6600
557,1.0,7137281959,6200
424,1.0,6865730150,9900
1074,1.0,6223969256,7600
1818,1.0,6223189532,10100
1058,1.0,6069788569,9300
1909,1.0,5481912001,4000
1033,1.0,5481850671,9100


In [122]:
# create df with rows where Crash Avg < 1
node_ids__crash_avg_census_tracts_clean = node_ids__crash_avg_census_tracts.loc[node_ids__crash_avg_census_tracts['CRASH_AVG']<1,:]

# check shape
node_ids__crash_avg_census_tracts_clean.shape

(2072, 3)

In [124]:
# save csv with node ids, census tracts, and DUMMY DANGER metric
node_ids__crash_avg_census_tracts_clean.to_csv(path_or_buf='../../data/DUMMY_CRASH_AVG_FOR_CORRELATION_TESTS_node_id_census_tract_key.csv')